# Aerial Cactus Identification Competition
### This is simple CNN modeling code which gets ROC AUC 0.9999
### If you think this notebook is useful, Please Upvote!

- [Competition Link](https://www.kaggle.com/c/aerial-cactus-identification)
- [Modeling Reference Notebook Link](https://www.kaggle.com/bonhart/simple-cnn-on-pytorch-for-beginers)


In [ ]:
import pandas as pd

# Data Path
data_path = '/kaggle/input/aerial-cactus-identification/'

labels = pd.read_csv(data_path + 'train.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [ ]:
from zipfile import ZipFile

# Extract train image data
with ZipFile(data_path + 'train.zip') as zipper:
    zipper.extractall()
    
# Extract test image data
with ZipFile(data_path + 'test.zip') as zipper:
    zipper.extractall()

In [ ]:
import torch # Pytorch 
import random
import numpy as np

# Fix Seed value
def set_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

set_seed(10)

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from sklearn.model_selection import train_test_split

# Split train data, valid data
train, valid = train_test_split(labels, 
                                test_size=0.1,
                                stratify=labels['has_cactus'],
                                random_state=10)

In [ ]:
import cv2
from torch.utils.data import Dataset # Class for generating data

class ImageDataset(Dataset):
    # Initialization Method (Generator)
    def __init__(self, df, img_dir = './', transform=None):
        super().__init__() # Inherit Dataset's __init__() method
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
    
    # Dataset Size Return Method
    def __len__(self):
        return len(self.df)
    
    # Data return method corresponding to index (idx)
    def __getitem__(self, idx):
        img_path = self.img_dir + self.df.iloc[idx, 0] # Image File Path
        image = cv2.imread(img_path) # Read image file
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Image color correction
        label = self.df.iloc[idx, 1] # Image label (target value)
        # Transform image
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [ ]:
from torchvision import transforms # Module for image transform

# Transforms for train dataset
transform_train = transforms.Compose([transforms.ToPILImage(),
                                      transforms.Pad(32, padding_mode='symmetric'),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.RandomRotation(10),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                           std=[0.2, 0.2, 0.2])])
# Transforms for valid dataset
transform_test= transforms.Compose([transforms.ToPILImage(),
                                    transforms.Pad(32, padding_mode='symmetric'),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                         std=[0.2, 0.2, 0.2])])

In [ ]:
dataset_all = ImageDataset(df=labels, img_dir='train/', transform=transform_train)
dataset_valid = ImageDataset(df=valid, img_dir='train/', transform=transform_test)

In [ ]:
from torch.utils.data import DataLoader # Class for generating data loaders

loader_all = DataLoader(dataset=dataset_all, batch_size=32, 
                          shuffle=True)
loader_valid = DataLoader(dataset=dataset_valid, batch_size=32, 
                          shuffle=False)

In [ ]:
import torch.nn as nn # Neural Network Module
import torch.nn.functional as F # Frequently used functions in neural network modules

class Model(nn.Module):
    # Define Neural Network Layer
    def __init__(self):
        super().__init__() # Inherit method __init__() of nn.Module
        # First to fifth convolution, batch normalization, maximum pooling layer
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=32,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(32), # Batch Normalization
                                    nn.LeakyReLU(), # LeakyReLU Activation Function
                                    nn.MaxPool2d(kernel_size=2))

        self.layer2 = nn.Sequential(nn.Conv2d(in_channels=32, out_channels=64,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(64),
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        
        self.layer3 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=128,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(128),
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        
        self.layer4 = nn.Sequential(nn.Conv2d(in_channels=128, out_channels=256,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(256),
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        
        self.layer5 = nn.Sequential(nn.Conv2d(in_channels=256, out_channels=512,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(512),
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        # Average Pooling Layer
        self.avg_pool = nn.AvgPool2d(kernel_size=4) 
        # Fully-connected Layer
        self.fc1 = nn.Linear(in_features=512 * 1 * 1, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=2)

    # Define Feed Forward 
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.avg_pool(x)
        x = x.view(-1, 512 * 1 * 1) # Flatten
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [ ]:
model = Model().to(device)

model

In [ ]:
# Loss Function
criterion = nn.CrossEntropyLoss()

In [ ]:
# Optimizer
optimizer = torch.optim.Adamax(model.parameters(), lr=0.0001)

In [ ]:
epochs = 50 # Total Epoch

# Train as much as the total epoch.
for epoch in range(epochs):
    epoch_loss = 0 # Initialize Loss by Epoch
    # Repeat extracting data by mini batch size 'repeat times'
    for images, labels in loader_all:
        # Assign data images, label's mini batch to device
        images = images.to(device)
        labels = labels.to(device)
        
        # Initialize gradient of optimizer
        optimizer.zero_grad()
        # Use image data as input for neural network models to calculate output values
        outputs = model(images)
        # Calculate the loss values of output and labels using the loss function
        loss = criterion(outputs, labels)
        # Perform Backpropagation
        loss.backward()
        # Renewal of weight
        optimizer.step()
        
        epoch_loss += loss.item() # Add Loss in Current Batch
        
    print(f'epoch: [{epoch+1}/{epochs}], loss: {epoch_loss/len(loader_all):.4f}')

In [ ]:
from sklearn.metrics import roc_auc_score # ROC AUC Calculation Function

# Initialize a list containing predicted probability values and real values
true_list = []
preds_list = []

model.eval() # Set model to evaluation state

with torch.no_grad(): # Disable gradient calculation
    for images, labels in loader_valid:
        # Assign images, label data mini-batch to device
        images = images.to(device)
        labels = labels.to(device)
        # Use image data as input for neural network models to calculate output values
        outputs = model(images)
        preds = torch.softmax(outputs.cpu(), dim=1)[:, 1] # Predicted Probability Value
        true = labels.cpu() # True Value 
        # Add predicted probability values and real values to the list
        preds_list.extend(preds)
        true_list.extend(true)
        
    # Validation Data ROC AUC Score Calculation
    print(f'Valid Data ROC AUC : {roc_auc_score(true_list, preds_list):.4f}')

In [ ]:
dataset_test = ImageDataset(df=submission, img_dir='test/', transform=transform_test)
loader_test = DataLoader(dataset=dataset_test, batch_size=32, 
                         shuffle=False)

In [ ]:
model.eval() # Set model to evaluation state

preds = [] # Initialize variables for storing target prediction values

with torch.no_grad(): # Disable gradient calculation
    for images, _ in loader_test:
        # Assign data images' mini batch to device
        images = images.to(device)
        # Use image data as input for neural network models to calculate output values
        outputs = model(images)
        # Probability that the target value is 1 (predicted value)
        preds_part = torch.softmax(outputs.cpu(), dim=1)[:, 1].tolist()
        # Connecting pres_part to pres
        preds.extend(preds_part)

In [ ]:
submission['has_cactus'] = preds
submission.to_csv('submission.csv', index=False)

In [ ]:
submission

In [ ]:
import shutil

shutil.rmtree('./train')
shutil.rmtree('./test')

# Upvote is FREE 👍